# Домашнее задание №2.
*Екатерина Волошина, БКЛ-182*

**NB!** К сожалению, output не работает вместе с проверкой на PEP-8. В репрезитории есть копия этого файла с проверкой на PEP-8!

## Задание 1. Книга
В директории лежит "Маленький принц" А. Сент-Экзюпери, откроем его в питоне и сохраним в переменную **text**.

In [169]:
with open('mprinc.txt_Ascii.txt', 'r', encoding='utf-8') as file:
    text = file.read()

## Задание 2. MyStem
Для начала импортируем модуль json для дальнейшей работы с ним.

In [170]:
import json

Теперь импортируем MyStem из модуля pymystem3.

In [171]:
from pymystem3 import Mystem
m = Mystem()

Чистим текст от знаков препинания.

In [172]:
from string import punctuation
for i in punctuation:
    text = text.replace(i, '')
text = text.replace('\n', ' ')

Проводим разбор с помощью analyze и замеряем время.

In [173]:
%%time
ana = m.analyze(text)

Wall time: 2.86 s


Записываем результат анализа в файл json.

In [174]:
with open('mystem.json', 'w', encoding='utf-8') as file:
    json.dump(ana, file, ensure_ascii=False)

## Задание 3. PyMorphy
Импортируем из nltk.tokenize word_tokenize и обрабатываем с помощью него текст.

In [175]:
from nltk.tokenize import word_tokenize
words = word_tokenize(text)

Импортируем MorphAnalyzer из pymorphy2. 

In [176]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Создаем пустой список, куда будем добавлять результат анализа.

In [177]:
words_parse = []

Парсим текст, результат записываем в файл.

In [178]:
%%time
for word in words:
    words_parse.append(morph.parse(word))

Wall time: 4.28 s


Создаем пустой список, в который добавляем словари вида {токен : лемма, часть речи}.

In [179]:
dict_words = []
for word in words_parse:
    first = word[0]
    dict_w = {}
    dict_w[first.word] = first.normal_form, first.tag.POS
    dict_words.append(dict_w)

Записываем результат результат анализа в файл json.

In [180]:
with open('dict.json', 'w', encoding='utf-8') as file:
    json.dump(dict_words, file, ensure_ascii=False)

Создаем пустой список для задания со звездочкой. Записываем туда весь разбор в список словарей. В словарь записываем как ключ токен, как значение - список, содержащий весь разбор .tag

In [181]:
dict_words_2 = []
for word in words_parse:
    first = word[0]
    dict_2 = {}
    dict_2[first.word] = str(first.tag).split(',')
    dict_words_2.append(dict_2)

Записываем результат в файл json.

In [182]:
with open('dict_2.json', 'w', encoding='utf-8') as file:
    json.dump(dict_words_2, file, ensure_ascii=False)

## Задание 4. Части речи и топ глаголов и наречий.
Импортируем Counter для дальнейшего составления частотных словарей.

In [183]:
from collections import Counter

Создаем пустой список, куда записываем часть речи каждого токена, затем создаем частотный словарь на основе этого списка

In [184]:
POS_list = []
for i in dict_words:
    for value in i.values():
        POS_list.append(value[1])
counter_POS = Counter(POS_list)

### Ответ:
Части речи и их отношение к общему числу слов.

In [185]:
for key, value in counter_POS.items():
    print(key, ' - ', value/len(POS_list)*100)

NOUN  -  19.856712529512336
PRCL  -  7.164373524383294
ADJF  -  10.363917609704469
None  -  0.46405601237482696
CONJ  -  11.59325897582024
PREP  -  7.148090857282423
VERB  -  15.31384840836929
INFN  -  2.8901734104046244
NPRO  -  12.643491003826426
ADVB  -  7.595864202556379
ADJS  -  1.2130586990148986
NUMR  -  1.0827973622079297
GRND  -  0.4233493446226492
COMP  -  0.5617520149800538
PRED  -  0.8304160221444272
PRTS  -  0.2523813400635024
PRTF  -  0.24424000651306685
INTJ  -  0.3582186762191647


Создаем пустые списки для глаголов и наречий.

In [186]:
verb_list = []
adverb_list = []

Добавляем в списки леммы всех глаголов и наречий соответственно. 

In [187]:
for i in dict_words:
    for value in i.values():
        if value[1] == 'VERB':
            verb_list.append(value[0])
        elif value[1] == 'ADVB':
            adverb_list.append(value[0])

Создаем на основе списков частотне словари, упорядоченные по частотности.

In [188]:
verb_top = Counter(verb_list).most_common()
adverb_top = Counter(adverb_list).most_common()

### Ответ: 
Топ-20 глаголов:

In [189]:
for i in range(20):
    print(verb_top[i][0], ' - ', verb_top[i][1])

быть  -  154
сказать  -  128
мочь  -  51
спросить  -  36
стать  -  35
ответить  -  29
знать  -  28
хотеть  -  22
видеть  -  20
увидеть  -  19
говорить  -  19
подумать  -  19
жить  -  18
нарисовать  -  18
понять  -  16
смотреть  -  14
любить  -  14
понимать  -  13
прибавить  -  12
приручить  -  12


Топ-20 наречий:

In [190]:
for i in range(20):
    print(adverb_top[i][0], ' - ', verb_top[i][1])

очень  -  154
потом  -  128
совсем  -  51
тоже  -  36
тогда  -  35
уже  -  29
никогда  -  28
там  -  22
потому  -  20
где  -  19
точно  -  19
здесь  -  19
тут  -  18
уж  -  18
мало  -  16
опять  -  14
вдруг  -  14
всего  -  13
снова  -  12
всегда  -  12


## Задание 5. Топы биграмм и триграмм.

Создаем пустой список для лемм, добавляем туда леммы всех словоформ.

In [191]:
lemmas = []
for i in words_parse:
    first = i[0]
    lemmas.append(first.normal_form)

Импортируем из nltk.util bigrams для составления биграмм.
Создаем список биграмм, на его основе создаем частотный словарь, который упорядочиваем по частотности.

In [192]:
from nltk.util import bigrams
bigrams = list(bigrams(lemmas))
top_bi = Counter(bigrams).most_common()

Аналогично создаем частотный словарь триграмм.

In [193]:
from nltk.util import trigrams
trigrams = list(trigrams(lemmas))
top_tri = Counter(trigrams).most_common()

### Ответ:
Топ-25 биграмм

In [194]:
for i in range(25):
    print(i+1, ' '.join(top_bi[i][0]), ' - ', top_bi[i][1])

1 маленький принц  -  176
2 я не  -  38
3 и я  -  38
4 у я  -  36
5 сказать он  -  30
6 он не  -  24
7 это быть  -  23
8 сказать маленький  -  22
9 и он  -  21
10 я быть  -  19
11 ничто не  -  17
12 ты быть  -  17
13 потому что  -  17
14 но я  -  17
15 спросить маленький  -  17
16 не мочь  -  16
17 он быть  -  15
18 и не  -  15
19 что ты  -  15
20 но он  -  15
21 не знать  -  15
22 принц и  -  15
23 добрый день  -  15
24 что я  -  13
25 и весь  -  13


Топ-25 триграмм

In [195]:
for i in range(25):
    print(i+1, ' '.join(top_tri[i][0]), ' - ', top_tri[i][1])

1 сказать маленький принц  -  22
2 спросить маленький принц  -  17
3 маленький принц и  -  15
4 и маленький принц  -  11
5 подумать маленький принц  -  8
6 маленький принц он  -  7
7 я человек серьёзный  -  7
8 добрый день сказать  -  7
9 но он не  -  6
10 он не ответить  -  6
11 у я есть  -  6
12 маленький принц не  -  6
13 у ты быть  -  5
14 на мой планета  -  5
15 повторить маленький принц  -  5
16 промолвить маленький принц  -  5
17 маленький принц а  -  5
18 маленький принц я  -  5
19 это я не  -  4
20 на самый дело  -  4
21 удав снаружи и  -  4
22 и он опять  -  4
23 потому что он  -  4
24 такой же как  -  4
25 но маленький принц  -  4
